In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam, rmsprop, sgd
from keras.callbacks import EarlyStopping
import keras.backend as Ka
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import KFold
from numpy import array
from keras import optimizers
import pandas as pd

Using TensorFlow backend.


# 1. Importation et préparation des données

In [2]:
immo = pd.read_csv('../../data/ml/train_immo_100_more_k_12_10.csv')

In [3]:
immo['mois'] = immo['mois'] - 12

### On vire les variables dont on a pas besoin

In [4]:
immo2 = immo.copy()

In [5]:
immo2 = immo2.drop(columns=['month'])

In [6]:
immo2.sort_values(by=['id_bien','mois']).head(20)

,id_bien,year,valeur,mnth_sin,mnth_cos,mois,valeur-1,valeur-2,valeur-3,valeur-4,...,valeur_diff-1-2,valeur_diff-2-3,valeur_diff-3-4,valeur_diff-4-5,valeur_diff-5-6,valeur_diff-6-7,valeur_diff-7-8,valeur_diff-8-9,valeur_diff-9-10,valeur_diff-10-11
0,16,2015,209500.000000,0.000000e+00,1.000000e+00,1,209200.000000,264625.000000,228424.000000,210381.000000,...,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000,2550.000000,-11637.500000,-63529.166667
1,16,2015,174100.000000,5.000000e-01,8.660254e-01,2,209500.000000,209200.000000,264625.000000,228424.000000,...,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000,2550.000000,-11637.500000
2,16,2015,276600.000000,8.660254e-01,5.000000e-01,3,174100.000000,209500.000000,209200.000000,264625.000000,...,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000,2550.000000
3,16,2015,248566.666667,1.000000e+00,6.123234e-17,4,276600.000000,174100.000000,209500.000000,209200.000000,...,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000
4,16,2015,229313.333333,8.660254e-01,-5.000000e-01,5,248566.666667,276600.000000,174100.000000,209500.000000,...,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667
5,16,2015,148330.000000,5.000000e-01,-8.660254e-01,6,229313.333333,248566.666667,276600.000000,174100.000000,...,-19253.333333,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667
6,16,2015,194712.500000,1.224647e-16,-1.000000e+00,7,148330.000000,229313.333333,248566.666667,276600.000000,...,-80983.333333,-19253.333333,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000,1968.500000
7,16,2015,0.000000,-5.000000e-01,-8.660254e-01,8,194712.500000,148330.000000,229313.333333,248566.666667,...,46382.500000,-80983.333333,-19253.333333,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000,18043.000000
8,16,2015,225850.000000,-8.660254e-01,-5.000000e-01,9,0.000000,194712.500000,148330.000000,229313.333333,...,-194712.500000,46382.500000,-80983.333333,-19253.333333,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000,36201.000000
9,16,2015,215570.000000,-1.000000e+00,-1.836970e-16,10,225850.000000,0.000000,194712.500000,148330.000000,...,225850.000000,-194712.500000,46382.500000,-80983.333333,-19253.333333,-28033.333333,102500.000000,-35400.000000,300.000000,-55425.000000


In [7]:
max_valeur = immo2.valeur.max()
min_valeur = immo2.valeur.min()

In [8]:
database = immo2.values

Nous avons implémenter deux fonctions :
- one_hot_encode_database()
- et normalize_database()

La première fonction permet de transformer la variable cible en plusieurs variables binaires (format requis par Keras)
La deuxième permet de normaliser les variables entre 0 et 1.

### On normalise les données quantitatives

In [9]:

## One hot encode the columns of the database specified in list_id.
#
#  return a copy of the database, on hot encoded. 
def one_hot_encode_database(database, list_id):
    encoded_database = np.empty(shape=(database.shape[0],0), dtype=float)
    for id in range(database.shape[1]):
        if id in list_id:
            original_column = database[:, id]

            print(np.unique(original_column).shape)
            encoded_column = to_categorical(original_column)
            print(id)
            print(encoded_column.shape)
            encoded_database = np.column_stack((encoded_database,encoded_column))
        else:
            original_column = database[:, id]
            encoded_database = np.column_stack((encoded_database,original_column))
    return encoded_database

## Normalize between 0 and 1 each column of the database specified in list_id.
#
#  return a copy of the database, normalized. 
def normalize_database(database, list_id):
    encoded_database = database.copy()
    for  id in list_id :
        x = encoded_database[:, id]
        encoded_database[:, id] = (x-min(x))/(max(x)-min(x))
    return encoded_database


In [10]:
res = pd.DataFrame(database)
res.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,16.0,2015.0,209500.000000,0.000000,1.000000e+00,1.0,209200.000000,264625.0,228424.0,210381.0,...,-55425.000000,36201.0,18043.0,1968.5,6079.166667,-33666.666667,51750.000000,2550.000000,-11637.500000,-63529.166667
1,16.0,2015.0,174100.000000,0.500000,8.660254e-01,2.0,209500.000000,209200.0,264625.0,228424.0,...,300.000000,-55425.0,36201.0,18043.0,1968.500000,6079.166667,-33666.666667,51750.000000,2550.000000,-11637.500000
2,16.0,2015.0,276600.000000,0.866025,5.000000e-01,3.0,174100.000000,209500.0,209200.0,264625.0,...,-35400.000000,300.0,-55425.0,36201.0,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000,2550.000000
3,16.0,2015.0,248566.666667,1.000000,6.123234e-17,4.0,276600.000000,174100.0,209500.0,209200.0,...,102500.000000,-35400.0,300.0,-55425.0,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667,51750.000000
4,16.0,2015.0,229313.333333,0.866025,-5.000000e-01,5.0,248566.666667,276600.0,174100.0,209500.0,...,-28033.333333,102500.0,-35400.0,300.0,-55425.000000,36201.000000,18043.000000,1968.500000,6079.166667,-33666.666667


In [11]:
cols_index = set(range(database.shape[1])) 
numerical_cols_index = cols_index  - {0,2,3,4,5}
numerical_cols_index = list(numerical_cols_index)

In [12]:
database = normalize_database(database, numerical_cols_index)

In [13]:
immo_train = pd.DataFrame(database)
immo_train.columns = immo2.columns
immo_train.head()

,id_bien,year,valeur,mnth_sin,mnth_cos,mois,valeur-1,valeur-2,valeur-3,valeur-4,...,valeur_diff-1-2,valeur_diff-2-3,valeur_diff-3-4,valeur_diff-4-5,valeur_diff-5-6,valeur_diff-6-7,valeur_diff-7-8,valeur_diff-8-9,valeur_diff-9-10,valeur_diff-10-11
0,16.0,0.0,209500.000000,0.000000,1.000000e+00,1.0,0.002207,0.002792,0.002410,0.001436,...,0.499494,0.499979,0.607908,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947,0.499769
1,16.0,0.0,174100.000000,0.500000,8.660254e-01,2.0,0.002211,0.002207,0.002792,0.001559,...,0.499789,0.499494,0.607984,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947
2,16.0,0.0,276600.000000,0.866025,5.000000e-01,3.0,0.001837,0.002211,0.002207,0.001806,...,0.499600,0.499789,0.607603,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995
3,16.0,0.0,248566.666667,1.000000,6.123234e-17,4.0,0.002919,0.001837,0.002211,0.001428,...,0.500331,0.499600,0.607835,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163
4,16.0,0.0,229313.333333,0.866025,-5.000000e-01,5.0,0.002623,0.002919,0.001837,0.001430,...,0.499639,0.500331,0.607686,0.499987,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871


### Integer Encoding de ib_bien

In [14]:
immo_train2 = immo_train.copy()

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
immo_train2["id_bien"] = le.fit_transform(immo_train["id_bien"])

In [16]:
immo_train2.head()

,id_bien,year,valeur,mnth_sin,mnth_cos,mois,valeur-1,valeur-2,valeur-3,valeur-4,...,valeur_diff-1-2,valeur_diff-2-3,valeur_diff-3-4,valeur_diff-4-5,valeur_diff-5-6,valeur_diff-6-7,valeur_diff-7-8,valeur_diff-8-9,valeur_diff-9-10,valeur_diff-10-11
0,0,0.0,209500.000000,0.000000,1.000000e+00,1.0,0.002207,0.002792,0.002410,0.001436,...,0.499494,0.499979,0.607908,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947,0.499769
1,0,0.0,174100.000000,0.500000,8.660254e-01,2.0,0.002211,0.002207,0.002792,0.001559,...,0.499789,0.499494,0.607984,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947
2,0,0.0,276600.000000,0.866025,5.000000e-01,3.0,0.001837,0.002211,0.002207,0.001806,...,0.499600,0.499789,0.607603,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995
3,0,0.0,248566.666667,1.000000,6.123234e-17,4.0,0.002919,0.001837,0.002211,0.001428,...,0.500331,0.499600,0.607835,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163
4,0,0.0,229313.333333,0.866025,-5.000000e-01,5.0,0.002623,0.002919,0.001837,0.001430,...,0.499639,0.500331,0.607686,0.499987,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871


# 2. Construction du modèle

La fonction build_NN permet de construire et de compiler un réseau de neurones.

La fonction test_model permet fiter puis de tester/valider le modèle avec une 5 fold validation. Elle retourne des indicateurs de performances ( matrice de confusion et coefficient de kappa de cohen ).

Pour trouver le modèle final, il a fallu tester affiner les paramètres du modèle avec une méthode de "test and learn".

Voilà les paramètres qu'il a fallu ajuster:
- nombre de neurones par couche / nombre de couches
- learning rate
- epochs
- patience


In [17]:
immo_train2.head()

,id_bien,year,valeur,mnth_sin,mnth_cos,mois,valeur-1,valeur-2,valeur-3,valeur-4,...,valeur_diff-1-2,valeur_diff-2-3,valeur_diff-3-4,valeur_diff-4-5,valeur_diff-5-6,valeur_diff-6-7,valeur_diff-7-8,valeur_diff-8-9,valeur_diff-9-10,valeur_diff-10-11
0,0,0.0,209500.000000,0.000000,1.000000e+00,1.0,0.002207,0.002792,0.002410,0.001436,...,0.499494,0.499979,0.607908,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947,0.499769
1,0,0.0,174100.000000,0.500000,8.660254e-01,2.0,0.002211,0.002207,0.002792,0.001559,...,0.499789,0.499494,0.607984,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995,0.499947
2,0,0.0,276600.000000,0.866025,5.000000e-01,3.0,0.001837,0.002211,0.002207,0.001806,...,0.499600,0.499789,0.607603,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163,0.499995
3,0,0.0,248566.666667,1.000000,6.123234e-17,4.0,0.002919,0.001837,0.002211,0.001428,...,0.500331,0.499600,0.607835,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871,0.500163
4,0,0.0,229313.333333,0.866025,-5.000000e-01,5.0,0.002623,0.002919,0.001837,0.001430,...,0.499639,0.500331,0.607686,0.499987,0.499797,0.500110,0.500048,0.499993,0.500007,0.499871


In [18]:
nb_type_bien = immo_train2['id_bien'].drop_duplicates().shape[0]

In [19]:
nb_type_bien

4750

In [20]:
nb_features = immo_train2.shape[1]-3

In [21]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.layers import Flatten

kernel_initialiser = 'normal'

nb_features # contient le nombre de features sans le type de bien ( ie mois, lags et diffs )

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
# notre entrée ici est un simple entier donc shape = 1
main_input = Input(shape=(1,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=20, input_dim=nb_type_bien, input_length=1,embeddings_initializer=kernel_initialiser)(main_input)

#Argument must be a dense tensor: ((4750,), 512) - got shape [2], but wanted [2, 1].

auxiliary_input = Input(shape=(nb_features,), name='aux_input')

x = Flatten()(x)

x = keras.layers.concatenate([x, auxiliary_input])

x = Dense(30, activation='relu',kernel_initializer=kernel_initialiser)(x)

x = Dense(100, activation='relu',kernel_initializer=kernel_initialiser)(x)

x = Dense(100, activation='relu',kernel_initializer=kernel_initialiser)(x)

main_output = Dense(1, name='main_output', activation='relu',kernel_initializer='normal')(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])


Instructions for updating:
Colocations handled automatically by placer.


In [22]:
sgd = keras.optimizers.Adam(lr=0.001)
#sgd = keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=sgd, loss="mean_squared_logarithmic_error")

# 3. Walk Forward Validation

La validation du modèle est réalisé grace à une Walk Forward Validation. On entraine le modèle sur 10 années pour prédire l'année suivante et on déplace la fenêtre d'année en année

In [23]:
from sklearn.metrics import mean_squared_log_error
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [24]:
45-12

33

In [25]:
30-12

18

In [26]:
from sklearn.ensemble import RandomForestRegressor

mean_error = []
for mois in range(18,22):
    train = immo_train2[immo_train2['mois'] < mois]
    val = immo_train2[immo_train2['mois'] == mois]
    
    test_headline_data = val['id_bien']
    test_additional_data = val.drop(columns=['id_bien','valeur','mois'])
    test_labels = val['valeur']
    
    headline_data = train['id_bien']
    additional_data = train.drop(columns=['id_bien','valeur','mois'])
    labels = train['valeur']
    
    print("mois : " + str(mois+12))
    
    model.fit([headline_data, additional_data], [labels], epochs=4, batch_size=32)
    
    preds = np.squeeze(model.predict([test_headline_data, test_additional_data]))
    
    error = rmsle(test_labels, preds)
    print('Week %d - Error %.5f' % (mois+12, error))
    mean_error.append(error)
    print('Mean Error inter = %.5f' % np.mean(mean_error))
print('Mean Error Final = %.5f' % np.mean(mean_error))




mois : 30
Instructions for updating:
Use tf.cast instead.
Epoch 1/4
80750/80750 [==============================] - 4s 52us/step - loss: 16.3162
Epoch 2/4
80750/80750 [==============================] - 4s 48us/step - loss: 7.1621
Epoch 3/4
80750/80750 [==============================] - 4s 50us/step - loss: 6.8672
Epoch 4/4
80750/80750 [==============================] - 4s 48us/step - loss: 6.7482
Week 30 - Error 2.02923
Mean Error inter = 2.02923
mois : 31
Epoch 1/4
85500/85500 [==============================] - 4s 47us/step - loss: 6.5558
Epoch 2/4
85500/85500 [==============================] - 4s 49us/step - loss: 6.5293
Epoch 3/4
85500/85500 [==============================] - 5s 54us/step - loss: 6.5100
Epoch 4/4
85500/85500 [==============================] - 4s 52us/step - loss: 6.4930
Week 31 - Error 1.79007
Mean Error inter = 1.90965
mois : 32
Epoch 1/4
90250/90250 [==============================] - 6s 62us/step - loss: 6.3085
Epoch 2/4
90250/90250 [==============================]

In [33]:
pd.DataFrame(labels).describe()

,valeur
count,9.500000e+04
mean,2.082349e+05
std,3.068302e+05
min,0.000000e+00
25%,1.126931e+05
50%,1.671459e+05
75%,2.457614e+05
max,6.085667e+07


In [277]:
error = rmsle(test_labels, preds)

In [437]:
error

2.0023613562879437

In [394]:
type(test_labels.values)

numpy.ndarray

In [395]:
test_labels.values.shape

(4750,)

In [391]:
preds.shape

(4750, 1)

In [398]:
print(np.squeeze(preds))

[ 0.0084166  -0.00675052 -0.00555914 ... -0.00019156  0.00814748
  0.00096494]


In [386]:
print(preds)

[[ 0.0084166 ]
 [-0.00675052]
 [-0.00555914]
 ...
 [-0.00019156]
 [ 0.00814748]
 [ 0.00096494]]


In [ ]:
    xtr, xts = train.drop(['valeur','id_bien'], axis=1), val.drop(['valeur','id_bien'], axis=1)
    ytr, yts = train['valeur'].values, val['valeur'].values
    
    mdl = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0)
    mdl.fit(xtr, ytr)
    
    p = mdl.predict(xts)
    
    error = rmsle(yts, p)
    print('Week %d - Error %.5f' % (mois, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))